# Create forcing file list
Populates a text file with the names of the forcing files used as SUMMA input.

In [19]:
# modules
import os
from pathlib import Path
from shutil import copyfile
from datetime import datetime

#### Control file handling

In [20]:
# Easy access to control file folder
controlFolder = Path('../../../0_control_files')

In [21]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [22]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line and not line.startswith('#'):
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [23]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Find forcing location

In [24]:
# Forcing path
forcing_path = read_from_control(controlFolder/controlFile,'forcing_summa_path')

In [25]:
# Specify default path if needed
if forcing_path == 'default':
    forcing_path = make_default_path('forcing/4_SUMMA_input') # outputs a Path()
else:
    forcing_path = Path(forcing_path) # make sure a user-specified path is a Path()

#### Find where forcing file list needs to go

In [26]:
# Forcing file list path & name
file_list_path = read_from_control(controlFolder/controlFile,'settings_summa_path')
file_list_name = read_from_control(controlFolder/controlFile,'settings_summa_forcing_list')

In [27]:
# Specify default path if needed
if file_list_path == 'default':
    file_list_path = make_default_path('settings/SUMMA') # outputs a Path()
else:
    file_list_path = Path(file_list_path) # make sure a user-specified path is a Path()

In [28]:
# Make the folder if it doesn't exist
file_list_path.mkdir(parents=True, exist_ok=True)

#### Make the file

In [29]:
# Find a list of forcing files
_,_,forcing_files = next(os.walk(forcing_path))

In [30]:
# Sort this list
forcing_files.sort()

In [31]:
# Create the file list
with open(file_list_path / file_list_name, 'w') as f:
    for file in forcing_files:
        f.write(str(file) + "\n")

#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [32]:
# Set the log path and file name
logPath = file_list_path
log_suffix = '_make_forcing_file_list.txt'

In [33]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [34]:
# Copy this script
thisFile = '1_create_forcing_file_list.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

In [35]:
# Get current date and time
now = datetime.now()

In [36]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Generated forcing file list.']
    for txt in lines:
        file.write(txt) 